In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('dictionary.csv')


In [3]:
df

,ID,Word,Alternative,Definition
0,0,a,kin,"PARTICLE (emphasis, emotion, or confirmation)"
1,1,akesi,NaN,"NOUN non-cute animal; reptile, amphibian"
2,2,ala,NaN,"ADJECTIVE no, not, zero"
3,3,alasa,NaN,"VERB to hunt, forage"
4,4,ale,ali,"ADJECTIVE all; abundant, countless, bountiful,..."
...,...,...,...,...
115,115,wan,NaN,"ADJECTIVE unique, united NUMBER one"
116,116,waso,NaN,"NOUN bird, flying creature, winged animal"
117,117,wawa,NaN,"ADJECTIVE strong, powerful; confident, sure; e..."
118,118,weka,NaN,"ADJECTIVE absent, away, ignored"


In [4]:
syllables = ['0', '1', '2', '3']
space = ' '

In [5]:
# Define a function to convert to any base n
def convert_to_base_n(x, base):
    if x == 0:
        return '0'

    base_str = ''
    while x > 0:
        remainder = x % base
        base_str = str(remainder) + base_str
        x //= base
    
    return base_str

# Specify the base you want (e.g., base 3)
base_n = len(syllables)

# Add a new column with base-n indices
df[f'Base-{base_n} Index'] = df.index.map(lambda x: convert_to_base_n(x, base_n))

In [6]:
print(df.to_string())

      ID     Word  Alternative                                                                                                                        Definition Base-4 Index
0      0        a          kin                                                                                     PARTICLE (emphasis, emotion, or confirmation)            0
1      1    akesi          NaN                                                                                          NOUN non-cute animal; reptile, amphibian            1
2      2      ala          NaN                                                                                                           ADJECTIVE no, not, zero            2
3      3    alasa          NaN                                                                                                              VERB to hunt, forage            3
4      4      ale          ali             ADJECTIVE all; abundant, countless, bountiful, every, plentiful NOUN abundance, everyth

In [7]:
import numpy as np
from scipy.io.wavfile import write

# Define a function to generate a sine wave for a given frequency and duration
def generate_sine_wave(frequency, duration_ms, amplitude=1.0, sample_rate=44100):
    t = np.linspace(0, duration_ms / 1000, int(sample_rate * (duration_ms / 1000)), endpoint=False)
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return wave

# Define the number of notes in an octave and their frequencies (adjust as needed)
num_notes = len(syllables)
base_frequency = 440.0  # A4, the 4th A on a piano
octave = [base_frequency * 2**(i/12) for i in range(num_notes)]

# Define the duration of each note in milliseconds (adjust as needed)
note_duration_ms = 500

# Generate and save audio files for each note
for note_index, frequency in enumerate(octave):
    note_wave = generate_sine_wave(frequency, note_duration_ms)
    write(f'note_{note_index}.wav', 44100, note_wave.astype(np.float32))

In [8]:
!pip3 install pydub

In [9]:
from pydub import AudioSegment
import numpy as np

def generate_silence(duration_ms, sample_width=2):
    # Generate silence using numpy
    sample_rate = 44100  # Adjust as needed
    num_samples = int(sample_rate * duration_ms / 1000)
    silence = np.zeros(num_samples, dtype=np.int16)
    
    # Convert the numpy array to an AudioSegment with the specified sample width
    audio_segment = AudioSegment(
        silence.tobytes(),
        frame_rate=sample_rate,
        sample_width=sample_width,
        channels=1
    )
    
    return audio_segment

def generate_sine_wave(frequency, duration_ms, sample_width=2):
    # Generate a sine wave using numpy
    sample_rate = 44100  # Adjust as needed
    t = np.linspace(0, duration_ms / 1000, int(sample_rate * duration_ms / 1000), endpoint=False)
    sine_wave = 32767 * np.sin(2 * np.pi * frequency * t)
    
    # Convert the numpy array to an AudioSegment with the specified sample width
    audio_segment = AudioSegment(
        sine_wave.tobytes(),
        frame_rate=sample_rate,
        sample_width=sample_width,
        channels=1
    )
    
    return audio_segment

def create_sound_from_base_n(base_n_string, n, output_file):
    # Define the frequencies for each note in an octave
    octave_frequencies = [440 * (2 ** (i / n)) for i in range(n)]
    
    # Create a mapping from digits to frequencies
    digit_to_frequency = {str(i): octave_frequencies[i] for i in range(n)}
    
    # Define the sample width (2 bytes for 16-bit audio)
    sample_width = 4
    
    # Create a list of audio segments for each digit in the base-n string
    audio_segments = []
    for digit in base_n_string:
        if digit == ' ':
            # Pause for the duration of a note
            audio_segments.append(generate_silence(duration_ms=100, sample_width=sample_width))  # 500 milliseconds pause
        else:
            # Play the note corresponding to the digit
            frequency = digit_to_frequency.get(digit, 0)  # Use 0 Hz for unsupported digits
            audio_segment = generate_sine_wave(frequency, duration_ms=100, sample_width=sample_width)  # 500 milliseconds per note
            audio_segments.append(audio_segment)
    
    # Concatenate the audio segments
    final_audio = sum(audio_segments)
    
    # Export the audio as a sound file
    final_audio.export(output_file, format="wav")

# Example usage
base_n_string = "10 20211"
n = 3
output_file = "output.wav"
create_sound_from_base_n(base_n_string, n, output_file)


/home/codespace/.python/current/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [10]:
def word_to_index(word):
    lines = !cat dictionary.csv | grep \,{word}\,
    line = lines[0]
    index = line.split(",")[0]
    return index 

def word_to_base_n(word):
    return df.iloc[int(word_to_index(word))][f"Base-{len(syllables)} Index"]

word_to_base_n('pona')

'1110'

In [11]:
text = "mama meli o awen."
text = text.replace(",", " ")
text = text.replace(":", " ")
text = text.replace(".", "  ")
text

'mama meli o awen  '

In [12]:
text_base_n = ' '.join([(word_to_base_n(word) if word else word) for word in text.split(" ")])
text_base_n

'301 303 1010 20  '

In [13]:
n = 3
output_file = "output.wav"
create_sound_from_base_n(text_base_n, n, output_file)